In [1]:
#  导入必要包
%matplotlib inline
#import utilities
import os
import shutil
import numpy as np
import random
from tqdm import tqdm  
from time import time
from PIL import Image
import h5py
import pandas as pd

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.callbacks import *
from keras.optimizers import *
from keras.utils import *
from keras import backend as K

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# 建立符号链接
def data_symbol_link():
    work_dir  = os.getcwd()
    train_dir = work_dir + "/train/"
    test_dir  = work_dir + "/test/"
    data_dir  = work_dir + "/data/"
    
    if(os.path.exists(data_dir)):
        shutil.rmtree(data_dir)
        
    split_train_dir = work_dir+"/data/train"
    split_test_dir  = work_dir+"/data/test"
    os.mkdir(data_dir)
    
    os.mkdir(split_train_dir)
    os.mkdir(split_train_dir+"/dog")
    os.mkdir(split_train_dir+"/cat")
    os.mkdir(split_test_dir)
    os.mkdir(split_test_dir+"/test")
        
    train_files = os.listdir(train_dir)    
    num_train_files = len(train_files)
    for i in tqdm(range(num_train_files)):
        file = train_files[i]
        if "dog" in file.split('.'):
            os.symlink(train_dir+file, split_train_dir+"/dog/"+file)
        else:
            os.symlink(train_dir+file, split_train_dir+"/cat/"+file)
    
    test_files = os.listdir(test_dir)    
    num_test_files = len(test_files)
    for i in tqdm(range(num_test_files)):
        file = test_files[i]
        os.symlink(test_dir+file, split_test_dir+"/test/"+file)
        
    return split_train_dir, split_test_dir    

In [3]:
train_data, test_data = data_symbol_link()

100%|██████████| 12500/12500 [00:00<00:00, 98511.49it/s]


In [4]:
# 融合模型
def write_feature_data(MODEL, image_shape, train_data, test_data, batch_size, preprocess_input = None):
    input_tensor = Input((image_shape[0], image_shape[1], 3))
    x = input_tensor
    if preprocess_input:
        x = Lambda(preprocess_input)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    base_model.save_weights(f'{base_model.name}-imagenet.h5')
    
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(train_data, image_shape, shuffle=False, 
                                              batch_size=batch_size)
    test_generator = gen.flow_from_directory(test_data, image_shape, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
    train_feature = model.predict_generator(train_generator, train_generator.samples, verbose=1)
    test_feature = model.predict_generator(test_generator, test_generator.samples, verbose=1)
    with h5py.File(f"feature_{base_model.name}.h5") as h:
        h.create_dataset("train", data=train_feature)
        h.create_dataset("test", data=test_feature)
        h.create_dataset("label", data=train_generator.classes)

In [5]:
# Xception
write_feature_data(Xception, (299, 299), train_data, test_data, batch_size=1, preprocess_input=xception.preprocess_input)

83689472/83683744 [==============================] - 1s 0us/step
Found 24961 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
12500/12500 [==============================] - 117s 9ms/step


In [6]:
# Densenet
write_feature_data(DenseNet201, (224, 224), train_data, test_data, batch_size=1, preprocess_input=densenet.preprocess_input)

76398592/76391848 [==============================] - 1s 0us/step
Found 24961 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
12500/12500 [==============================] - 343s 27ms/step


In [7]:
# InceptionV3
write_feature_data(InceptionV3, (299, 299), train_data, test_data, batch_size=1, preprocess_input=inception_v3.preprocess_input)

87916544/87910968 [==============================] - 1s 0us/step
Found 24961 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
12500/12500 [==============================] - 188s 15ms/step


In [4]:
# 构建网络
from sklearn.utils import shuffle
np.random.seed(42)

X_train = []
X_test = []

for filename in ["feature_xception.h5", "feature_densenet201.h5", "feature_inception_v3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        Y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, Y_train = shuffle(X_train, Y_train)

In [5]:
# 建立顶层网络结构
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [7]:
#训练模型并保存顶层网络参数
filepath="merged_weight.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]
model.fit(X_train, Y_train, batch_size=128, epochs=20, validation_split=0.2, shuffle=True,
         callbacks=callbacks_list)
model.save_weights("merged_weights.h5")

Train on 19968 samples, validate on 4993 samples
Epoch 1/20
19968/19968 [==============================] - 2s 87us/step - loss: 0.0643 - acc: 0.9817 - val_loss: 0.0168 - val_acc: 0.9962

Epoch 00001: val_loss improved from inf to 0.01683, saving model to merged_weight.h5
Epoch 2/20
19968/19968 [==============================] - 1s 37us/step - loss: 0.0178 - acc: 0.9946 - val_loss: 0.0125 - val_acc: 0.9964

Epoch 00002: val_loss improved from 0.01683 to 0.01252, saving model to merged_weight.h5
Epoch 3/20
19968/19968 [==============================] - 1s 37us/step - loss: 0.0141 - acc: 0.9956 - val_loss: 0.0114 - val_acc: 0.9966

Epoch 00003: val_loss improved from 0.01252 to 0.01144, saving model to merged_weight.h5
Epoch 4/20
19968/19968 [==============================] - 1s 37us/step - loss: 0.0130 - acc: 0.9958 - val_loss: 0.0115 - val_acc: 0.9968

Epoch 00004: val_loss did not improve from 0.01144
Epoch 5/20
19968/19968 [==============================] - 1s 37us/step - loss: 0.0116

In [8]:
# 在测试集预测
model.load_weights('merged_weights.h5')
y_test = model.predict(X_test, verbose=1)
y_test = y_test.clip(min=0.005, max=0.995)

12500/12500 [==============================] - 0s 32us/step


In [9]:
df = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(test_data, (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_test[i])

df.to_csv('submission_merged.csv', index=None)
df.head(10)

Found 12500 images belonging to 1 classes.


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005
